In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

In [75]:
df=pd.read_csv('cardekho_cleaned.csv')
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15411 entries, 0 to 15410
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   model              15411 non-null  object 
 1   vehicle_age        15411 non-null  int64  
 2   km_driven          15411 non-null  int64  
 3   seller_type        15411 non-null  object 
 4   fuel_type          15411 non-null  object 
 5   transmission_type  15411 non-null  object 
 6   mileage            15411 non-null  float64
 7   engine             15411 non-null  int64  
 8   max_power          15411 non-null  float64
 9   seats              15411 non-null  int64  
 10  selling_price      15411 non-null  int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 1.3+ MB


#### Train and Test split

In [77]:
X=df.drop(columns=["selling_price"],axis=1)
Y=df['selling_price']

In [78]:
from sklearn.model_selection  import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=.3,random_state=42)

In [79]:
classification_features=[feature for feature in X.columns if(X[feature].dtype == 'object')]
classification_features

['model', 'seller_type', 'fuel_type', 'transmission_type']

In [80]:
numerical_features=[feature for feature in X.columns if(X[feature].dtype != 'object')]
numerical_features

['vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power', 'seats']

#### LabourEncoding

In [81]:
from sklearn.preprocessing import LabelEncoder
for feature in classification_features:
	labour=LabelEncoder()
	labour.fit(X[feature])
	X_train[feature]=labour.transform(X_train[feature])
	X_test[feature]=labour.transform(X_test[feature])

#### Apply Standardization

In [82]:
from sklearn.preprocessing import StandardScaler
for feature in classification_features:
	scaler=StandardScaler()
	scaler.fit(X_train[[feature]])
	X_train[feature]=scaler.transform(X_train[[feature]])
	X_test[feature]=scaler.transform(X_test[[feature]])

In [83]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [84]:
# Evaluate the model
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [85]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "Adaboost Regressor":AdaBoostRegressor(),
	"GradientBoost Regressor":GradientBoostingRegressor()
}

In [86]:
for model_name,model in models.items():
    model.fit(X_train, Y_train) 

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(Y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(Y_test, y_test_pred)

    
    print(model_name)
    
    print(f'Model performance for Training set')
    print(f"- Root Mean Squared Error: {model_train_rmse}")
    print(f"- Mean Absolute Error: {model_train_mae}")
    print(f"- R2 Score: {model_train_r2}")

    print('----------------------------------')
    
    print(f'Model performance for Test set')
    print(f"- Root Mean Squared Error: {model_test_rmse}")
    print(f"- Mean Absolute Error: {model_test_mae}")
    print(f"- R2 Score: {model_test_r2}")
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 560327.6024971509
- Mean Absolute Error: 268807.6923638746
- R2 Score: 0.6169165412402811
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 508110.8542847031
- Mean Absolute Error: 281212.38836442895
- R2 Score: 0.6567089015518186


Lasso
Model performance for Training set
- Root Mean Squared Error: 560327.6025047041
- Mean Absolute Error: 268807.6891540316
- R2 Score: 0.6169165412299531
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 508110.88257054705
- Mean Absolute Error: 281212.31382857997
- R2 Score: 0.6567088633307154


Ridge
Model performance for Training set
- Root Mean Squared Error: 560327.6025189749
- Mean Absolute Error: 268807.30871053383
- R2 Score: 0.6169165412104399
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 508111.1745126955
- Mean Absolute Error: 281211

In [ ]:
GradientBoostingRegressor()

In [92]:
rf_params = {"max_depth": [5, 8, 15, None, 10],
             "max_features": [5, 7, "auto", 8],
             "min_samples_split": [2, 8, 15, 20],
             "n_estimators": [100, 200, 500, 1000]}

adaboost_params={
                "n_estimators":[10,50,100,150],
				"learning_rate": [1.0,2.0,1.5,2.5],
				"loss":['linear', 'square', 'exponential']
                }

gradientboost_params={
	"n_estimators":[50,60,70,80,90],
	"loss":['squared_error', 'absolute_error', 'huber', 'quantile'],
	"criterion":['friedman_mse','squared_error','mse'],
	"min_samples_split": [2, 8, 15, 20],
	"max_depth": [5, 8, 15, None, 10]
}

In [93]:
randomcv_models=[
	("Random Forest Regressor", RandomForestRegressor(),rf_params),
	("Adaboost Regressor",AdaBoostRegressor(),adaboost_params),
	("GradientBoost Regressor",GradientBoostingRegressor(),gradientboost_params)
]

In [94]:
from sklearn.model_selection import RandomizedSearchCV

models_best_params={}
for name,model,params in randomcv_models:
	random=RandomizedSearchCV(estimator=model,param_distributions=params,cv=5)

	random.fit(X_train,Y_train)
	models_best_params[name]=random.best_params_

for model_name,best_params in models_best_params.items():
	print(f"---------------- Best Params for {model_name} -------------------")
	print(best_params)

---------------- Best Params for Random Forest Regressor -------------------
{'n_estimators': 500, 'min_samples_split': 2, 'max_features': 8, 'max_depth': None}
---------------- Best Params for Adaboost Regressor -------------------
{'n_estimators': 10, 'loss': 'linear', 'learning_rate': 1.0}
---------------- Best Params for GradientBoost Regressor -------------------
{'n_estimators': 60, 'min_samples_split': 2, 'max_depth': 15, 'loss': 'huber', 'criterion': 'squared_error'}


In [95]:
models = {
    "Random Forest Regressor": RandomForestRegressor(n_estimators=500,min_samples_split=2,max_features=8,max_depth=None),
	"Adaboost Regressor":AdaBoostRegressor(n_estimators=10,loss="linear",learning_rate=1.0),
	"GradientBoost Regressor":GradientBoostingRegressor(n_estimators=60,min_samples_split=2,max_depth=15,loss='huber',criterion='squared_error')
}

In [96]:
for model_name,model in models.items():
    model.fit(X_train, Y_train) 

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(Y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(Y_test, y_test_pred)

    
    print(model_name)
    
    print(f'Model performance for Training set')
    print(f"- Root Mean Squared Error: {model_train_rmse}")
    print(f"- Mean Absolute Error: {model_train_mae}")
    print(f"- R2 Score: {model_train_r2}")

    print('----------------------------------')
    
    print(f'Model performance for Test set')
    print(f"- Root Mean Squared Error: {model_test_rmse}")
    print(f"- Mean Absolute Error: {model_test_mae}")
    print(f"- R2 Score: {model_test_r2}")
    
    print('='*35)
    print('\n')

Random Forest Regressor
Model performance for Training set
- Root Mean Squared Error: 133488.57214106972
- Mean Absolute Error: 39208.057164815866
- R2 Score: 0.9782580976247836
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 239383.42027884687
- Mean Absolute Error: 103773.35065645019
- R2 Score: 0.9238037263051183


Adaboost Regressor
Model performance for Training set
- Root Mean Squared Error: 356145.27464193327
- Mean Absolute Error: 235952.68095509434
- R2 Score: 0.8452382118590873
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 408879.99822140037
- Mean Absolute Error: 255952.1830276341
- R2 Score: 0.7777010918741849


GradientBoost Regressor
Model performance for Training set
- Root Mean Squared Error: 76266.14697286449
- Mean Absolute Error: 12754.052941459518
- R2 Score: 0.992903029857706
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 289522.